<a href="https://colab.research.google.com/github/OksanaTymchenko/BudgetCheck/blob/master/BudgetCheck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q docx2python
import numpy as np
import pandas as pd
import re
from docx2python import docx2python
from functools import reduce

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
budget_doc = docx2python('/content/drive/My Drive/Colab Notebooks/budget.docx')
budget = list(filter(lambda x: len(x) > 1, budget_doc.body))
budget = budget[1:9]


In [0]:
def clean_parts(raw_budget):
    temp = pd.DataFrame(raw_budget[1:])
    temp.drop(temp.columns[:-2], axis=1, inplace=True)
    temp.dropna(inplace=True)
    temp.columns = ['Правки', 'Висновки']
    # temp.dropna(subset = ['Правки'], inplace=True)
    temp = temp[temp.iloc[:,0].map(lambda d: len(d[0]) > 0)]
    print(temp.head())
    return temp


In [0]:
budget_df = list(map(clean_parts, budget))
budget_df = pd.concat(budget_df, ignore_index = True)

In [0]:
def edit_split(edit):
    if edit[0] and edit[0].startswith('-'):
      number, first_author = edit[0].split(maxsplit = 1)
      author_list = [first_author]
      k = 1
      while 'Н.д.' in edit[k]:
        author_list.append(edit[k])
        k += 1
      text = '\n'.join(edit[len(author_list):])
      return number, author_list, text
    else: 
      return None

In [0]:
def get_edit_number(number):
  return re.search('\d+', number).group()

In [0]:
# a, b, c = edit_split(budget_df.loc[439]['Правки'])
def get_deputies(dep_list):
  try:
    deputies = []
    grp = False
    if re.match('\s*(Комітет|Фракція)', dep_list[0]):
      grp = True
      deputies.append({'deputy':dep_list[0], 'card':None})
      dep_list = dep_list[1:]
    for dep in dep_list:
      res = re.search(r"Н.д.([^(]+).*№(\d+)", dep)
      if res:
       deputies.append({'deputy':res.group(1).strip(), 'card':res.group(2)})
      else:
        res = re.search(r".([^(]+).*№(\d+)", dep)
        deputies.append({'deputy':res.group(1).strip(), 'card':res.group(2)})
    if len(deputies) > 1:
      grp = True
    return deputies, grp
  except:
    print('depError', dep_list)
# get_deputies(b)

In [0]:
def get_action(edit):
  # try:
    keywords = ['(збільшити|передбачити)(.*?)(видатк(и|ах)|субвенці(ю|ї)|дотаці(ю|ї))',
                # '((збільши(ти|вши))|передбачити|встановити).*(видатк(и|ів)|субвенці(ю|ї)|дотаці(ю|ї)|обсяг|кошт(и|ів))',
                # -зменшивши!!!
                # '(встановити) .*збільшивши',
                '(включити|відновити|передбачити)(.*?)бюджетну програму',
                '(встановити)(.*?)(видатки|субвенці(ю|ї))',
                '(збільшити)(.*?)обсяг (асигнувань|видатків)',
                '(встановити|(від|по)новити)(.*?)субвенці(ю|ї)',
                '(доповнити)(.*?)програм(ою|ами)',
                '(збільшити|відновити) фінансування', '(включити) (((в|у) роз)|до)', 
                '(Передбачити) нову програму',
                '(передбачити|виділити)(.*?)[^«]кошти',
                '(надати) додатково',
                '^(?:видатки|субвенцію)(.*?)(збільшити)',
                '^(визначити) обсяг',
                'видатки (встановити)',
                '(передбачити) збільшення(.*?)видатків',
                # '(встановити) .*тобто збільшити', 
                '(збільшення) видаткової частини', 
                # '(?:видатк(?:и|ів)|субвенці(?:ю|ї)|дотаці(?:ю|ї)).*(збільшити|передбачити|визначити|встановити)', 
                # '^встановити видатки'
                '(передбачити) \d+',
                '(передбачити) програму',
                '^додаткову дотацію(.*?)(збільшити)',
                '(забезпечити) (встановлення обсягів видатків|додаткове фінансування)'
                ]
    action_list = []

    for keyword in keywords:
      sentence = '[^\.]*'+ keyword
      act = re.compile(sentence, re.IGNORECASE)
      res = act.search(edit)
      if res:
        edit = edit[res.start():]
        if re.search('\d+', edit) and 'зменшивши' not in edit:
          return res

    return None
          # return res.group(1)


    # for keyword in keywords:
    #     action_list.append(re.compile(keyword, re.IGNORECASE))

    # # uah = re.compile('грн|грив(ень|ні|ня)')
    # for act in action_list:
    #   res = act.search(edit_text)
    #   if res:

    #     if re.search('\d+', edit_text) and 'зменшивши' not in edit_text:
    #         return res.group(1)
    
  # except:
  #   print('actionError', edit_text)


In [0]:
s = """збільшити видатки на 100 грн для цього зменшити видатки на 200 грн"""
print(get_action(s).group(0))

збільшити видатки


In [0]:
def find_price(text):
  money_str = '(?P<amount>[\d.,]+)\s?(?P<order>тис|млн|млрд?|тисяч)?\.?\s?(грн|грив(ень|ні|ня))'
  s_increase = '(збільш(ити|ивши|ення))(.*?)на (суму )?' + money_str
  price = re.search(s_increase, text, re.IGNORECASE)
  # price = re.findall(s_increase, text, re.IGNORECASE)
  if price:
    return format_sum(price.group('amount'), price.group('order'))
  else:
    return None
  # s_prep = '((в|у) обсязі|(в|у) сумі|(в|у) розмірі|до|обсягом)?'
  # s_comp = re.compile(s_prep + '\s'+ money_str)
  # return s.search(text).group()


In [0]:
print(find_price(s))

[('збільшити', 'ити', ' видатки ', '', '100', '', 'грн', '')]
None


In [0]:
def get_sum(edit):
    money = re.search('([\d.,]+)([а-я. ]*)(грн|грив(ень|ні|ня))', edit)
    if money:
      return money
      # return money.group(1), money.group(2)
    else:
      return None

In [0]:

def format_sum(amount, order):
  # order = money_match[1].strip(' .')
  # try:
    amount_str = float(amount.replace(' ','').replace('.', '').replace(',','.'))
    switcher = {
        'тис': amount_str*10**3,
        'млн': amount_str*10**6,
        'млрд': amount_str*10**9,
        'тисяч': amount_str*10**3,
        'млр': amount_str*10**9
    }
    return switcher.get(order, amount_str)
  # except:
  #   return 0


In [0]:
def get_code(edit):
    code = re.compile('(за кодом|КПКВК|по коду|код|за кодами) (\d+)', re.IGNORECASE)
    # res = code.findall(edit)
    res = code.search(edit)
    if res:
      # code = res.group(2)
      # if code[0]=='0':
      #   code = code[1:]
      return res.group(2)
    #   if len(res)>1:
    #   for r in res:
    #     code = r[1]
    #     if code[0]==0:
    #       code = code[1:]
   
    return None

In [0]:
s = "033883"
if s[0]=='0':
  print("!")

!


In [0]:
def get_percent(edit):
  try:
    percent = re.search('[^\.\n]*[\d\.,]+\s*(%|відсотк(ів|ка))[^\.\n]*', edit)
    # percent = re.search(' [\d\.\,]+\s*(%|відсотків|відсотка) ', edit)
    if percent:
      return percent.group(0)
      # before = edit[:percent.start()].rsplit(maxsplit = 2)
      # print(before)
      # after = edit[percent.end():].split(maxsplit = 3)
      # print(after)
      # return ' '.join(before[1:]) + percent.group() + ' '.join(after[:3]) 
    else:
      return None
  except:
    print('percentError', edit)

# get_percent("фінансове забезпечення наукової діяльності та законодавчо гарантованих виплат науковим працівникам відповідно до статті 48 Закону України «Про наукову та науково-технічну діяльність» у розмірі не менше 1,7% валового внутрішнього продукту")

In [0]:
print(re.search('[^\.,]*', "nkd,f.n"))

<_sre.SRE_Match object; span=(0, 3), match='nkd'>


In [0]:
def get_level(edit):
  level = re.search('на рівні (.*?)[\d.,%?]+', edit)
  if 'на рівні' in edit:
    return 'на рівні'
  else:
    return None
  

In [0]:
def get_at_expense(edit):
  exp = re.compile('((за рахунок)|джерелом|джерелами|(шляхом.*зменшення)).*\.', re.DOTALL | re.IGNORECASE)
  exp = exp.search(edit)
  compensated = False
  if exp:
    if 'зменшення' in exp.group():
      compensated = True 
    return exp, compensated
  else:
    return None


In [0]:
def get_sums(edit):
    money_str = '([\d\.,]+)\s?([а-я]{,5})?\.?\s?(грн|грив(ень|ні|ня))'
    # money_str = '([\d.,]+)\s?(тис|млн|млрд?|тисяч)?\.?\s?(грн|грив(ень|ні|ня))'
    money_sums = re.findall(money_str, edit)
    if money_sums and len(money_sums) == 1:
      return format_sum(money_sums[0][0], money_sums[0][1])
    elif len(money_sums) > 1:
      includes_regex = re.compile(money_str + '.*((у|в) тому числі|з яких).*' + money_str, re.DOTALL)
      if includes_regex.search(edit):
        # print(test_regex.search(edit).group())
        return format_sum(money_sums[0][0], money_sums[0][1])
      else:
        first = format_sum(money_sums[0][0], money_sums[0][1])
        rest = sum([ format_sum(s[0], s[1]) for s in money_sums])
        if first == rest:
          return first
        else: 
          return first + rest
      # res = sum(list(map(format_sum, sums)))
      # return res
    return None

In [0]:
# money_str = '([\d.,]+)\s?(тис|млн|млрд?|тисяч)?\.?\s?(грн|грив(ень|ні|ня))'
# r = re.compile(money_str)
# for index, row in budget_df.iterrows():
#     edit_parts = edit_split(row['Правки'])
#     if edit_parts:
#       raw_number, deputies_list, text = edit_parts
#       action = get_action(text)
#       if action:
#         res = r.findall(text)
#         for i in res:
#           if i[1]=='': print(i)
        

In [0]:
res = pd.DataFrame()
for index, row in budget_df.iterrows():
    edit_parts = edit_split(row['Правки'])
    if edit_parts:
      raw_number, deputies_list, text = edit_parts
      action = get_action(text)
      if action:
        action_start = action.start()
        action_end = action.end()
        action = action.group(1)
        # text = text[action.start():]
        at_exp = get_at_expense(text[action_end:])
        end_lookup = len(text)
        if at_exp:
          end_lookup = action_end + at_exp[0].start()
          at_exp = at_exp[0].group()
        edit_part = text[action_start : end_lookup]
        # print(raw_number)
        # print(edit_part)
        price = find_price(edit_part)
        # print(price)
        # code = get_code(edit_part)
        
       

[('збільшити', 'ити', ' ', '', '1,2', 'млрд', 'грн', ''), ('збільшити', 'ити', ' ', '', '5', 'млрд', 'грн', '')]
[('Збільшити', 'ити', ' за загальним фондом видатки за кодом 0501020 ', '', '1.476.400', 'тис', 'грн', ''), ('збільшення', 'ення', ' фінансування видатків Служби судової охорони, в тому числі: видатки споживання - ', '', '1.240.200', 'тис', 'грн', '')]
[('збільшити', 'ити', ': освітню субвенцію ', '', '6.100', 'тис', 'грн', ''), ('збільшити', 'ити', ' базову дотацію ', '', '110.500', 'тис', 'грн', '')]
[('Збільшити', 'ити', ' освітню субвенцію ', '', '680.973,8', 'тис', 'грн', ''), ('Збільшити', 'ити', ' видатки за кодом 2211230 ', '', '200', 'млн', 'грн', '')]
[('збільшити', 'ити', ' видатки розвитку ', '', '500', 'млн', 'грн', ''), ('збільшити', 'ити', ' видатки розвитку ', '', '5,5', 'млрд', 'грн', '')]


In [65]:
edits = pd.DataFrame(columns = ['Номер правки', 'Що зробити', 'Депутат', 
                                'Реєстраційна картка №', 'Правка', 'На рівні',
                                'Відсоток від', 'Ціна правки', 'Сума в бюджеті',
                                'Хотєлка', 'Код правки', 'Категорія', 'За рахунок',
                                'Видалити/коментар', 'Статус', 'Повний тескт']) 
group_edits = edits.copy()
comp_edits = edits.copy()
for index, row in budget_df.iterrows():
  try:
    edit_parts = edit_split(row['Правки'])
    if edit_parts:
      raw_number, deputies_list, text = edit_parts
      action =  get_action(text)
      if action:
        # whole_text = text
        action_start = action.start()
        action_end = action.end()
        action = action.group(1)
        # text = text[action.start():]
        at_exp = get_at_expense(text[action_end:])
        compensated = False
        end_lookup = len(text)
        if at_exp:
          end_lookup = action_end + at_exp[0].start()
          # edit_part = text[action_start:action_end + exp_start]
          compensated = at_exp[1]
          at_exp = at_exp[0].group()
        edit_part = text[action_start : end_lookup]
        percent = get_percent(edit_part)
        level = get_level(edit_part)
        price = find_price(edit_part)
        if price:
          money = price
        else:
          money = get_sums(edit_part)
        if money or percent or level:
          number = get_edit_number(raw_number)
          deputies, group = get_deputies(deputies_list)
          code = get_code(edit_part)
          for deputy in deputies:
            edit_dict = {'Номер правки':number, 'Що зробити':action, 'Правка':text,
                    'Депутат':deputy['deputy'], 'Реєстраційна картка №':deputy['card'],
                    'Хотєлка':money, 'Код правки':code, 'Відсоток від':percent,
                    'Ціна правки': price,
                    'На рівні':level, 'За рахунок':at_exp, 
                    'Статус':row['Висновки'][0], 'Повний тескт':'\n'.join(row['Правки'])}
          # edit_dict = {'number':number, 'action':action, 
          #              'deputy':deputy['deputy'], 'card':deputy['card'],
          #               'sum':money, 'code':code,
          #              'percent':percent, 'level':level, 'exp':exp, 
          #              'status':row['Висновки'][0], 'text':row['Правки']}
          # edit_dict = {'Номер правки':number, 'Що зробити':action, 
          #             'Депутат':deputy['deputy'], 'Реєстраційна картка №':deputy['card'],
          #             'Хотєлка':money, 'Код правки':code,
          #             'Відсоток від':percent, 'На рівні':level, 'За рахунок':exp, 
          #             'Статус':row['Висновки'][0], 'Повний тескт':row['Правки']}
        
          # edits = edits.append(edit_dict, ignore_index=True)
            if compensated:
              comp_edits = comp_edits.append(edit_dict, ignore_index=True)
            elif group:
              group_edits = group_edits.append(edit_dict, ignore_index=True)
            else:
              edits = edits.append(edit_dict, ignore_index=True)
  except:
    print(raw_number, text)
print(len(edits), len(group_edits), len(comp_edits))

602 4444 630


In [0]:
with pd.ExcelWriter('/content/drive/My Drive/budget_test.xlsx') as writer: 
  edits.to_excel(writer, sheet_name='Хотєлки', index = False)
  group_edits.to_excel(writer, sheet_name='Групові', index = False)
  comp_edits.to_excel(writer, sheet_name='Компенсовані', index = False)
# edits.to_excel('/content/drive/My Drive/budget_test.xlsx', index = False)


-1423- збільшення для Волинської області обсягів освітньої субвенції на 1,6 млрд грн, медичної субвенції на 24.076,2 тис.грн. 
-1429- Районний бюджет Криворізького району (код 04305200000) внести такі зміни:
Освітня субвенція - цифри «86441,8» замінити на «103730,16»;
Медична субвенція - цифри «8109,6» замінити на «9731,52». 

субвенцію визначити у розмірі 37.868,1 тис. грн, збільшивши відповідно на 16.401 тис. грн; 

«Субвенція ...
 «Освітня субвенція з державного бюджету місцевим бюджетам» у підсумковому рядку суму 77.533.320,8 тис.грн замінити на суму 105.533.321 тис. грн.

за кодом видатки встановити
 
Внести зміни до загальної суми субвенції спеціального фонду на фінансове забезпечення будівництва, реконструкції, ремонту і 

Внести зміни до

у обсязі | в сумі | в розмірі

Установити, що у 2020 році кошти Державного фонду регіонального розвитку (КПКВ 2761070) у сумі 500.000 тис. гривень спрямовуються 

кошти Державного фонду регіонального розвитку (КПКВ 2761070) у сумі 500.000 тис. гривень спрямовуються 

передбачається сума у розмірі 960.000 тис. гривень,
збільшити обсяг асигнувань

Впровадження програми реформування та розвитку енергетичного сектору (Код 2401630) збільшити з 25.721 тис. грн. до 55.721 тис. грн. (за рахунок коштів, що надходять від акцизу за електроенергію).
в тому числі - 47
у тому числі - 75
